In [1]:
import numpy as np
import torch
from torchvision.datasets import mnist
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor

In [2]:
import time
batch_size=256
test_dataset = mnist.MNIST(root='../../test', train=False, transform=ToTensor())
test_loader = DataLoader(test_dataset, batch_size=batch_size)
def calc():
    all_correct_num = 0
    all_sample_num = 0
    model.eval()
    t_ls=[]
    fps_ls=[]
    for idx, (test_x, test_label) in enumerate(test_loader):
        start = time.time()
        predict_y = model(test_x.float()).detach()
        end = time.time()
        total_time = (end - start)
        fps=1/(total_time)
        fps_ls.append(fps)
        t_ls.append(total_time)
        predict_y = np.argmax(predict_y, axis=-1)
        current_correct_num = predict_y == test_label
        all_correct_num += np.sum(current_correct_num.numpy(), axis=-1)
        all_sample_num += current_correct_num.shape[0]
    t=sum(t_ls)
    fps=sum(fps_ls)/len(fps_ls)
    avg_time=t/len(t_ls)
    acc = all_correct_num / all_sample_num
    return t,avg_time,fps,acc

In [3]:
model=torch.load("mnist_0.99.pkl")
calc()

(0.7101612091064453, 0.017754030227661134, 81.83874632893446, 0.9906)

In [4]:
from torchsummary import summary
summary(model,(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 25, 25]             170
              ReLU-2           [-1, 10, 25, 25]               0
         MaxPool2d-3           [-1, 10, 12, 12]               0
            Conv2d-4             [-1, 22, 8, 8]           5,522
              ReLU-5             [-1, 22, 8, 8]               0
         MaxPool2d-6             [-1, 22, 4, 4]               0
            Linear-7                  [-1, 120]          42,360
              ReLU-8                  [-1, 120]               0
            Linear-9                   [-1, 84]          10,164
             ReLU-10                   [-1, 84]               0
           Linear-11                   [-1, 10]             850
             ReLU-12                   [-1, 10]               0
Total params: 59,066
Trainable params: 59,066
Non-trainable params: 0
---------------------------------

In [8]:
s=0
a=0
for i in range(100):
    temp=calc()
    s+=temp[1]
    a+=temp[2]
print(s/100,a/100)

7746.436803454538 0.9906000000000005


In [9]:
s=0
a=0
s1=0
for i in range(100):
    temp=calc()
    s1+=temp[0]
    s+=temp[1]
    a+=temp[2]
print(s1/100,s/100,a/100)

1.2826747465133668 7803.0628196147945 0.9906000000000005


In [10]:
1/(1.2826747465133668/40)

31.184834743749406